In [1]:
#라이브러리 충돌 해결 코드
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
from pdf2image import convert_from_path
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import ImageFont, ImageDraw, Image, ImageTk
import easyocr

In [4]:
conda install -c conda-forge poppler 

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda




In [3]:
# easyocr
reader = easyocr.Reader(['ko','en'])
result = reader.readtext("sample.jpg", width_ths=0.1, adjust_contrast=0.1)
for i in result[:100]:
    print(i)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


([[866, 184], [1008, 184], [1008, 246], [866, 246]], '김해틀', 0.5560641991150652)
([[1018, 168], [1232, 168], [1232, 261], [1018, 261]], '아름답게', 0.9560057520866394)
([[1235, 172], [1393, 172], [1393, 249], [1235, 249]], '시멘올', 0.5080405616959007)
([[1407, 178], [1624, 178], [1624, 254], [1407, 254]], '행복하게"', 0.9980339445042559)
([[1181, 235], [1282, 235], [1282, 342], [1181, 342]], '해', 0.9999737741282502)
([[319, 368], [376, 368], [376, 431], [319, 431]], '우', 0.9998619604067613)
([[387, 367], [621, 367], [621, 429], [387, 429]], '621-010', 0.9999618816480752)
([[661, 367], [778, 367], [778, 432], [661, 432]], '경남', 0.99364655835725)
([[786, 366], [954, 366], [954, 439], [786, 439]], '김해시', 0.9962421106244084)
([[961, 364], [1132, 364], [1132, 439], [961, 439]], '부원동', 0.9894853219914868)
([[1138, 367], [1246, 367], [1246, 429], [1138, 429]], '623', 0.9999972469798912)
([[1254, 374], [1303, 374], [1303, 428], [1254, 428]], '1', 0.9493452144455752)
([[1305, 365], [1416, 365], [1416, 432

In [7]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk, ImageDraw

result = []

class App:
    def __init__(self, master):
        global result
        
        # 이미지 열기
        self.src = cv2.imread('sample.jpg')
        img_w, img_h, img_a = self.src.shape # image size(height, width, alpha)

        self.master = master
        self.canvas = tk.Canvas(master, width=int(img_w * 0.25), height=int(img_h * 0.5))
        self.canvas.pack()

        self.src = cv2.resize(self.src, (int(img_w * 0.25), int(img_h * 0.5)))
        reader = easyocr.Reader(['ko','en'])
        result = reader.readtext(self.src, width_ths=0.1, adjust_contrast=0.1, low_text=0.1)
        
        img = Image.fromarray(self.src)    # 이미지에 그리기 위해서 CV2 이미지를 PIL 이미지 객체로 변환
        # font = ImageFont.truetype('./assets/gulim.ttf',15)
        draw = ImageDraw.Draw(img)

        np.random.seed(42)
        COLORS = np.random.randint(0, 255, size=(255, 3),dtype="uint8")

        for i in result :
            # 이미지 좌표 변수 지정
            x = i[0][0][0] 
            y = i[0][0][1] 
            w = i[0][1][0] - i[0][0][0] 
            h = i[0][2][1] - i[0][1][1]
            
            # 랜덤 색상 지정
            color_idx = random.randint(0,254)
            color = [int(c) for c in COLORS[color_idx]]

            # 이미지에 인식 영역 / 인식 문자 출력
            draw.rectangle(((x, y), (x+w, y+h)), outline=tuple(color), width=1)
            # draw.text((int((x + x + w) / 2) , y-2),str(i[1]), font=font, fill=tuple(color),)
        self.image = img
        self.image_tk = ImageTk.PhotoImage(self.image)

        # 이미지를 Canvas 위젯에 배치
        self.canvas.create_image(0, 0, anchor="nw", image=self.image_tk)

        # 클릭 이벤트 바인딩
        self.canvas.bind("<Button-1>", self.on_click)

    def on_click(self, event):
        global result

        # 클릭된 위치 계산
        input_x, input_y = event.x, event.y

        # 모자이크 처리
        for i in result:
            x = i[0][0][0] 
            y = i[0][0][1] 
            w = i[0][1][0]
            h = i[0][2][1]
            if x <= input_x <= w and y <= input_y <= h:
                draw = ImageDraw.Draw(self.image)
                draw.rectangle((x, y, w, h), fill=(0, 0, 0))
        # 모자이크 처리된 이미지를 다시 Tkinter PhotoImage로 변환하여 업데이트
        self.image_tk = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, anchor="nw", image=self.image_tk)

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
